<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/LDA_CrossValidation_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [211]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

In [212]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
df = pd.read_csv("/content/drive/MyDrive/datasets/OnlineNewsPopularity.csv").reset_index(drop = True)

In [214]:
df.head(3)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.5,-0.1875,0.0,0.1875,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.0,0.0000,0.5,0.0000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.0,0.0000,0.5,0.0000,1500


In [215]:
[x.strip() for x in df.columns]

['url',
 'timedelta',
 'n_tokens_title',
 'n_tokens_content',
 'n_unique_tokens',
 'n_non_stop_words',
 'n_non_stop_unique_tokens',
 'num_hrefs',
 'num_self_hrefs',
 'num_imgs',
 'num_videos',
 'average_token_length',
 'num_keywords',
 'data_channel_is_lifestyle',
 'data_channel_is_entertainment',
 'data_channel_is_bus',
 'data_channel_is_socmed',
 'data_channel_is_tech',
 'data_channel_is_world',
 'kw_min_min',
 'kw_max_min',
 'kw_avg_min',
 'kw_min_max',
 'kw_max_max',
 'kw_avg_max',
 'kw_min_avg',
 'kw_max_avg',
 'kw_avg_avg',
 'self_reference_min_shares',
 'self_reference_max_shares',
 'self_reference_avg_sharess',
 'weekday_is_monday',
 'weekday_is_tuesday',
 'weekday_is_wednesday',
 'weekday_is_thursday',
 'weekday_is_friday',
 'weekday_is_saturday',
 'weekday_is_sunday',
 'is_weekend',
 'LDA_00',
 'LDA_01',
 'LDA_02',
 'LDA_03',
 'LDA_04',
 'global_subjectivity',
 'global_sentiment_polarity',
 'global_rate_positive_words',
 'global_rate_negative_words',
 'rate_positive_words',
 

In [216]:
df.columns = [x.strip() for x in df.columns]

In [217]:
#Este bien podría ser un problema de regresión pero lo haremos de clasificación utilizando un punto de corte en la variable númerica para hacerlo de clasificación
# Al hacer df["shares"] > 6000 nos devuelve una lista booleana que al multiplicarla por 1 los TRUE serán 1 y los FALSE 0
df["success"] = (df["shares"] > 6000)*1

In [218]:
#Verificamos el cambios
df["success"].unique()

array([0, 1])

In [219]:
#Veamos la distibución del target
df['success'].value_counts(1)

0    0.896504
1    0.103496
Name: success, dtype: float64

In [220]:
ls_cont = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']
target = "shares"
target_disc = "success"

In [221]:
X = df[ls_cont]
yr = df[target]
yc = df[target_disc]

In [222]:
#Partimos data frame
X_train, X_test, y_train, y_test = train_test_split(X, yc)

In [223]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((29733, 44), (9911, 44), (29733,), (9911,))

In [224]:
X_train.head(3)

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
30673,13.0,328.0,0.625000,1.0,0.734884,8.0,2.0,1.0,0.0,5.158537,7.0,-1.0,2300.0,605.200000,0.0,843300.0,225985.714286,0.0,3496.268184,1694.647485,536.0,5500.0,3018.0,0.885645,0.028574,0.028573,0.028573,0.028635,0.496477,0.218068,0.045732,0.021341,0.681818,0.318182,0.409091,0.100000,1.0,-0.196429,-0.50,-0.05,0.0,0.000000,0.5,0.000000
33845,9.0,173.0,0.660606,1.0,0.793814,5.0,3.0,2.0,0.0,4.502890,10.0,-1.0,643.0,232.111111,0.0,843300.0,253020.000000,0.0,8936.470588,3416.710250,2900.0,2900.0,2900.0,0.020987,0.020154,0.136973,0.020219,0.801666,0.489610,0.158766,0.040462,0.023121,0.636364,0.363636,0.424675,0.136364,1.0,-0.137500,-0.15,-0.10,0.5,0.136364,0.0,0.136364
30905,10.0,751.0,0.484438,1.0,0.655773,3.0,1.0,2.0,1.0,4.949401,3.0,-1.0,458.0,228.500000,0.0,843300.0,281252.666667,0.0,3490.551453,1316.183818,458.0,458.0,458.0,0.066680,0.066681,0.733292,0.066675,0.066673,0.400339,0.088362,0.031957,0.015979,0.666667,0.333333,0.328492,0.050000,0.6,-0.238657,-0.50,-0.10,0.0,0.000000,0.5,0.000000


## Análisis Discriminante 

In [225]:
#Instanceamos el modelo
lda = LinearDiscriminantAnalysis()

In [226]:
#Entrenamos el modelo
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001)

# Cross Validation

In [227]:
#Hacemos cross validation indicano que queremos 4 pliegues
ls_res = cross_val_score(estimator = lda, X=X_train, y=y_train, cv=4, n_jobs=-1, scoring="roc_auc")

In [228]:
ls_res

array([0.70532546, 0.6962179 , 0.69057582, 0.69469995])

In [229]:
#Evaluemos el rendimiento del modelo en cuanto a sesgo y varianza
np.mean(ls_res), np.std(ls_res)

(0.6967047818225072, 0.005388338921325572)

In [230]:
#Ahora metamos las variables escaladas para verificar que las magnitudes no causen un menor performance
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
Xs = sc.fit_transform(X_train)
Xt = sc.fit_transform(X_test)

In [231]:
#Observamos que el escalamiento no ayuda, seguiremos con la escala que ya tienen los datos
ls_res = cross_val_score(estimator = lda, X=Xs, y=y_train, cv=4, n_jobs=-1, scoring="roc_auc")
np.mean(ls_res), np.std(ls_res)

(0.6967047818225072, 0.005388338921325572)

In [188]:
LinearDiscriminantAnalysis?

# Busqueda con Grid Search CV

In [232]:
lda.get_params()

{'n_components': None,
 'priors': None,
 'shrinkage': None,
 'solver': 'svd',
 'store_covariance': False,
 'tol': 0.0001}

In [233]:
#creamos rejilla
param_grid = {"solver": ["svd", "lsqr", "eigen"],
              "shrinkage": [None, "auto"] + [x/100 for x in range(2)]+[x for x in range(2)],
              "tol":[x/100 for x in range(2)]+[x for x in range(2)] }

In [234]:
param_grid

{'shrinkage': [None, 'auto', 0.0, 0.01, 0, 1],
 'solver': ['svd', 'lsqr', 'eigen'],
 'tol': [0.0, 0.01, 0, 1]}

In [235]:
list(map(len, param_grid.values()))

[3, 6, 4]

In [236]:
np.prod(list(map(len, param_grid.values())))

72

In [241]:
#Intanceamos el escalador
sc = MinMaxScaler()
sc.fit(X_train)
Xs = sc.transform(X_train)

In [242]:
#Rapidamente seleccionamor las mejores 5 variables de nuestro set con SelectKBest
kb = SelectKBest(k = 8, score_func=f_classif)
kb.fit(Xs, y_train)
Xb = kb.transform(Xs)
Xb

array([[2.63157895e-02, 7.81250000e-03, 2.76698667e-04, ...,
        3.57879758e-03, 3.10577563e-02, 3.08384881e-02],
       [1.64473684e-02, 1.56250000e-02, 2.76698667e-04, ...,
        3.43887110e-03, 1.48884340e-01, 2.18221565e-02],
       [9.86842105e-03, 1.56250000e-02, 2.76698667e-04, ...,
        5.43104471e-04, 7.97057093e-01, 7.19619709e-02],
       ...,
       [4.93421053e-02, 7.03125000e-02, 2.76698667e-04, ...,
        1.84987549e-02, 3.62333414e-02, 4.45143327e-01],
       [9.53947368e-02, 1.95312500e-01, 2.76698667e-04, ...,
        1.52733310e-01, 2.71781127e-02, 9.71248794e-01],
       [3.94736842e-02, 1.56250000e-02, 2.76698667e-04, ...,
        9.81264082e-04, 3.38264842e-01, 2.15874791e-02]])

In [243]:
kb.get_support()

array([False, False, False, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False, False,  True, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [244]:
#Nos qudamos con las variables que el modelo nos indica que sirven
ls_best = [x for x, y in zip(ls_cont, kb.get_support()) if y]
ls_best


['num_hrefs',
 'num_imgs',
 'kw_min_avg',
 'kw_max_avg',
 'kw_avg_avg',
 'self_reference_avg_sharess',
 'LDA_02',
 'LDA_03']

In [245]:
#Veamos los valores que le dio a cada variable
pd.DataFrame(zip(ls_cont, kb.scores_)).sort_values(by = 1)

,0,1
11,kw_min_min,0.000865
24,LDA_01,0.004291
2,n_unique_tokens,0.149810
3,n_non_stop_words,0.243739
0,n_tokens_title,0.282279
4,n_non_stop_unique_tokens,0.312715
33,rate_negative_words,0.343134
30,global_rate_positive_words,0.550437
6,num_self_hrefs,0.607167
1,n_tokens_content,0.785236


In [246]:
#Definimos la busqueda mediante 'GridSearchCV' buscara por cada parametro indicado
rs = GridSearchCV(cv=4, error_score=-1000, estimator=lda, n_jobs=-1,
                  scoring="roc_auc", param_grid=param_grid, verbose=True)

In [247]:
#Iniciamos el entrenamiento de los modelos
rs.fit(X=Xb, y=y_train)

Fitting 4 folds for each of 72 candidates, totalling 288 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 288 out of 288 | elapsed:    6.6s finished


GridSearchCV(cv=4, error_score=-1000,
             estimator=LinearDiscriminantAnalysis(n_components=None,
                                                  priors=None, shrinkage=None,
                                                  solver='svd',
                                                  store_covariance=False,
                                                  tol=0.0001),
             iid='deprecated', n_jobs=-1,
             param_grid={'shrinkage': [None, 'auto', 0.0, 0.01, 0, 1],
                         'solver': ['svd', 'lsqr', 'eigen'],
                         'tol': [0.0, 0.01, 0, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=True)

In [248]:
#Por medio de ".best_score_" podemos encontrar 
rs.best_score_

0.6838588202239281

In [249]:
pd.DataFrame(zip(ls_best, rs.best_estimator_.coef_[0])).sort_values(by = 1)

,0,1
3,kw_max_avg,-26.985636
2,kw_min_avg,-0.693212
6,LDA_02,-0.270421
7,LDA_03,0.298626
1,num_imgs,1.136075
0,num_hrefs,3.557857
5,self_reference_avg_sharess,4.407556
4,kw_avg_avg,33.580820


# Set de prueba

In [250]:
#Escalamos las variables con lo aprendido del set de entrenamiento
Xt=sc.transform(X_test)

In [251]:
#Transformamos el set escalado de prueba con lo aprendido del KbEST
kb.transform(Xt) 

array([[6.25000000e-02, 0.00000000e+00, 2.76698667e-04, ...,
        3.23728211e-02, 1.63216416e-01, 7.00771471e-01],
       [1.97368421e-02, 0.00000000e+00, 1.16213440e-02, ...,
        1.00794498e-02, 2.17439039e-02, 8.99190360e-02],
       [1.31578947e-02, 0.00000000e+00, 2.76698667e-04, ...,
        2.72738053e-03, 2.71746963e-02, 2.71946027e-02],
       ...,
       [2.30263158e-02, 7.81250000e-03, 4.73677373e-01, ...,
        1.42298115e-03, 7.32945531e-02, 7.19602559e-02],
       [0.00000000e+00, 0.00000000e+00, 6.79672543e-01, ...,
        0.00000000e+00, 3.10842759e-02, 9.55202119e-01],
       [2.96052632e-02, 7.81250000e-03, 6.58483444e-01, ...,
        0.00000000e+00, 6.28260654e-01, 2.40704348e-02]])

In [252]:
#Predecimos
predicciones=rs.predict(kb.transform(Xt))

In [253]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pd.Series(predicciones))

0.5050840235890912